In [ ]:
import glob
import numpy as np 
import time
import math
import random
from scipy import linalg as LA
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from termcolor import colored
import os
import pickle
from datetime import datetime
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# install package

In [ ]:
pip install trjtrypy

In [ ]:
# to make a directory
%cd 'gdrive/My Drive/traj-dist'

/content/gdrive/My Drive/traj-dist


In [ ]:
# to see in what directory we are in

In [ ]:
pwd

'/content/gdrive/My Drive/traj-dist'

In [ ]:
# to install setup.py from the current directory
!python setup.py install

In [ ]:
!pip install geohash2

# "######################" Now restart runtime "######################"

But don't run the above block

In [ ]:
import traj_dist.distance as tdist
import pickle

# Import data

## Preprocessing

In [ ]:
I = glob.glob('/content/gdrive/My Drive/Research/T-drive for 2ed paper/Chosen users for 5 pairs/*.txt',
              recursive=True)

In [ ]:
I1 = [0] * len(I)
for j in range(len(I)):
    I1[j] = int(I[j][81:85])
I1.sort()
I1

[1350, 1950, 2876, 3142, 3260, 5896, 5970, 6168, 6834, 9513]

In [ ]:
pairs_from_data = [[3142, 6834], [6168, 9513], [1950, 5896], [2876, 3260], [1350, 5970]]
pairs_from_data

[[3142, 6834], [6168, 9513], [1950, 5896], [2876, 3260], [1350, 5970]]

In [ ]:
Dict = {1350:0, 1950:1, 2876:2, 3142:3, 3260:4, 5896:5, 5970:6, 6168:7, 6834:8, 9513:9}
Dict

{1350: 0,
 1950: 1,
 2876: 2,
 3142: 3,
 3260: 4,
 5896: 5,
 5970: 6,
 6168: 7,
 6834: 8,
 9513: 9}

In [ ]:
pairs = np.array([[Dict[pairs_from_data[i][0]], Dict[pairs_from_data[i][1]]] \
                  for i in range(len(pairs_from_data))])
pairs

array([[3, 8],
       [7, 9],
       [1, 5],
       [2, 4],
       [0, 6]])

In [ ]:
J = [0] * len(I)
K = [0] * len(I)
for i in range(len(I)):
    for j in range(len(I)):
        if int(I[j][81:85]) == I1[i]:
            J[i] = I[j][:81]+str(i)+'.txt'
            K[i] = I[j]
J

['/content/gdrive/My Drive/Research/T-drive for 2ed paper/Chosen users for 5 pairs/0.txt',
 '/content/gdrive/My Drive/Research/T-drive for 2ed paper/Chosen users for 5 pairs/1.txt',
 '/content/gdrive/My Drive/Research/T-drive for 2ed paper/Chosen users for 5 pairs/2.txt',
 '/content/gdrive/My Drive/Research/T-drive for 2ed paper/Chosen users for 5 pairs/3.txt',
 '/content/gdrive/My Drive/Research/T-drive for 2ed paper/Chosen users for 5 pairs/4.txt',
 '/content/gdrive/My Drive/Research/T-drive for 2ed paper/Chosen users for 5 pairs/5.txt',
 '/content/gdrive/My Drive/Research/T-drive for 2ed paper/Chosen users for 5 pairs/6.txt',
 '/content/gdrive/My Drive/Research/T-drive for 2ed paper/Chosen users for 5 pairs/7.txt',
 '/content/gdrive/My Drive/Research/T-drive for 2ed paper/Chosen users for 5 pairs/8.txt',
 '/content/gdrive/My Drive/Research/T-drive for 2ed paper/Chosen users for 5 pairs/9.txt']

In [ ]:
K

['/content/gdrive/My Drive/Research/T-drive for 2ed paper/Chosen users for 5 pairs/1350.txt',
 '/content/gdrive/My Drive/Research/T-drive for 2ed paper/Chosen users for 5 pairs/1950.txt',
 '/content/gdrive/My Drive/Research/T-drive for 2ed paper/Chosen users for 5 pairs/2876.txt',
 '/content/gdrive/My Drive/Research/T-drive for 2ed paper/Chosen users for 5 pairs/3142.txt',
 '/content/gdrive/My Drive/Research/T-drive for 2ed paper/Chosen users for 5 pairs/3260.txt',
 '/content/gdrive/My Drive/Research/T-drive for 2ed paper/Chosen users for 5 pairs/5896.txt',
 '/content/gdrive/My Drive/Research/T-drive for 2ed paper/Chosen users for 5 pairs/5970.txt',
 '/content/gdrive/My Drive/Research/T-drive for 2ed paper/Chosen users for 5 pairs/6168.txt',
 '/content/gdrive/My Drive/Research/T-drive for 2ed paper/Chosen users for 5 pairs/6834.txt',
 '/content/gdrive/My Drive/Research/T-drive for 2ed paper/Chosen users for 5 pairs/9513.txt']

In [ ]:
taxi_id = np.array([J[i][81:-4] for i in range(len(J))])
I0 = np.sort(list(map(int, taxi_id)))
True in I0 == list(map(int, taxi_id)) # so the taxi ids are nn.arange(len(J))

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [ ]:
# create 'daysDate' function to convert start and end time to a float number of days

def days_date(time_str):
    date_format = "%Y-%m-%d %H:%M:%S"
    current = datetime.strptime(time_str, date_format)
    date_format = "%Y-%m-%d"
    bench = datetime.strptime('2006-12-30', date_format)
    no_days = current - bench
    delta_time_days = no_days.days + current.hour / 24.0 + current.minute / (24. * 60.) + current.second / (24. * 3600.)
    return delta_time_days

days_date = np.vectorize(days_date)
float1 = np.vectorize(float)

In [ ]:
def read_file(file_name):
    data = []
    f = open(file_name, "r")
    for line in f:
        item = line.strip().split(",")
        if len(item) == 4:
            data.append(np.asarray(item[1:]))
    data = np.array(data)
    return data

## Remove stationary points

In [ ]:
def remove_segments(traj): # removes stationary points
    p2 = traj[:,1:][1:]
    p1 = traj[:,1:][:-1]
    L = ((p2-p1)*(p2-p1)).sum(axis =1)
    I = np.where(L>1e-16)[0]
    return traj[I]

### In data1 below:
        (1) there is no stationary points
        (2) there is no trajectories with less than 1 or more than 200 waypoints

In [ ]:
T = len(K)
data1 = [] 
taxi_ids = []
for i in range(T):
    a = read_file(K[i])
    if len(a) > 1000:  
        a[:,0] = days_date(a[:,0])
        a = float1(a)
        data1.append(remove_segments(a)) 
        taxi_ids.append(i)

data1 = np.array(data1, dtype='object')
I = np.where(np.array([len(data1[i]) for i in range(len(data1))]) > 0)[0]
data1 = data1[I]
taxi_ids = np.array(taxi_ids, dtype='object')
print("len(data1) =", len(data1))
print("selected taxi_ids: \n", taxi_ids[I])
print([len(data1[i]) for i in range(len(data1))])

len(data1) = 10
selected taxi_ids: 
 [0 1 2 3 4 5 6 7 8 9]
[3127, 4870, 2129, 3145, 2697, 4410, 4741, 3391, 2534, 2593]


In [ ]:
def partition(trajectory, threshold=20):
    trajectories = []
    a = 24 * 60 * (trajectory[-1][0] - trajectory[0][0])
    if a <= threshold:
        return np.array(trajectory.reshape(1, len(trajectory), 3))
    else: 
        i = 0
        while a > threshold:
            j = i + 0
            val = 0
            while val < threshold: 
                if i < len(trajectory) - 1:
                    temp = val + 0
                    val += 24 * 60 * (trajectory[:,0][1:][i] - trajectory[:,0][:-1][i])
                    i += 1
                else: 
                    break
            if len(trajectory[j:i-1]) > 0:
                trajectories.append(trajectory[j:i-1])
            a = a - val
        if len(trajectory[i:]) > 0:
            trajectories.append(trajectory[i:])
    trajectories = np.array(trajectories, dtype='object')
    return trajectories

## Get length of each taxi trajectories before filtering between 100 and 200

In [ ]:
data1_len = [0] * len(data1)
for i in range(len(data1)):
    count = 0
    A = partition(data1[i], threshold=20)
    for j in range(len(A)):
        if len(A[j]) >= 1:
            count += 1
    data1_len[i] = count

print(data1_len)

[153, 167, 124, 256, 222, 146, 150, 170, 249, 172]


### In data3 below 
    1) any taxi has between 100 and 200 trajectories
    2) in each taxi dataset any trajectory has less than 60 minutes time

In [ ]:
data3 = [[0]] * len(data1)
for i in range(len(data1)):
    data3[i] = []
    A = partition(data1[i], threshold=20)
    for j in range(len(A)):
        if ((len(A[j]) >= 10) and (len(A[j]) <= 200)):
            data3[i].append(A[j])
    data3[i] = np.array(data3[i], dtype='object')
    
data3 = np.array(data3, dtype='object')
I = np.where(np.array([len(data3[i]) for i in range(len(data3))]))[0]
data3 = data3[I]
print("len(data3) =", len(data3))

Lenn = np.array([len(data3[i]) for i in range(len(data3))])
J2 = np.where((Lenn >= 100) & (Lenn <= 200))[0]
A = np.array([len(data3[J2][i]) for i in range(len(data3[J2]))])
print(colored(f"min and max length of selected users: {min(A), max(A)}", "green"))

data = data3[J2]
taxi_idxs = taxi_ids[J2]
print("number of selected users:", len(data))
print(colored(f"selected taxi ids: \n {taxi_idxs}", "blue"))
print([len(data[i]) for i in range(len(data))])

len(data3) = 10
min and max length of selected users: (100, 143)
number of selected users: 10
selected taxi ids: 
 [0 1 2 3 4 5 6 7 8 9]
[132, 140, 100, 143, 132, 123, 127, 143, 116, 119]


In [ ]:
for i in range(len(pairs)):
    print("pairs =", pairs[i])
    print(colored(f'pair from data: {pairs_from_data[i]}', 'red'))
    print(colored(f"Lengths before preprocessing: {data1_len[pairs[i][0]], data1_len[pairs[i][1]]}", 'yellow'))
    print(colored(f"Lengths after preprocessing: {len(data[pairs[i][0]]), len(data[pairs[i][1]])}", 'blue'))
    print()

pairs = [3 8]
pair from data: [3142, 6834]
Lengths before preprocessing: (256, 249)
Lengths after preprocessing: (143, 116)

pairs = [7 9]
pair from data: [6168, 9513]
Lengths before preprocessing: (170, 172)
Lengths after preprocessing: (143, 119)

pairs = [1 5]
pair from data: [1950, 5896]
Lengths before preprocessing: (167, 146)
Lengths after preprocessing: (140, 123)

pairs = [2 4]
pair from data: [2876, 3260]
Lengths before preprocessing: (124, 222)
Lengths after preprocessing: (100, 132)

pairs = [0 6]
pair from data: [1350, 5970]
Lengths before preprocessing: (153, 150)
Lengths after preprocessing: (132, 127)



In [ ]:
data_with_time = data + 0

In [ ]:
for i in range(len(data)):
    for j in range(len(data[i])):
        data[i][j] = data[i][j][:,1:]

In [ ]:
Min = np.min([np.min([np.min(data[i][j], 0) for j in range(len(data[i]))], 0) for i in range(len(data))], 0)
Max = np.max([np.max([np.max(data[i][j], 0) for j in range(len(data[i]))], 0) for i in range(len(data))], 0)
Mean = np.mean([np.mean([np.mean(data[i][j], 0) for j in range(len(data[i]))], 0) for i in range(len(data))], 0)
Std = np.std([np.std([np.std(data[i][j], 0) for j in range(len(data[i]))], 0) for i in range(len(data))], 0)
Min, Mean, Max, Std, LA.norm(Max - Mean)

(array([101.6124 ,  23.44004]),
 array([116.4305318,  39.9366455]),
 array([117.376  ,  41.09412]),
 array([0.08017081, 0.10837572]),
 1.4945425171273097)

# From github page: https://github.com/bguillouet/traj-dist

## Generating distance matrices (Continuous Frechet)

In [ ]:
pairs

array([[3, 8],
       [7, 9],
       [1, 5],
       [2, 4],
       [0, 6]])

In [ ]:
# data = np.concatenate((data1, data2), 0)
def calculate_dists_line_by_line_frechet(data, pair_idx, start_idx, end_idx): 
    n = len(data)
    for i in range(start_idx, end_idx):
        start_time = time.time()
        def f(traj):
            return tdist.frechet(data[i], traj)
        A = list(map(f, data[i+1:]))
        np.savetxt('/content/gdrive/My Drive/traj-dist/Calculated Distance Matrices (T-drive)/frechet lines '+str(pair_idx)+'/'+str(i)+'.csv', 
                   A, delimiter=',')
        total_time = time.time() - start_time
        print(f"time for step {i}: {total_time}")
    return total_time

In [ ]:
for j in range(5):
    print(len(data[pairs[j][0]]) + len(data[pairs[j][1]]))

259
262
263
232
259


In [ ]:
#j = 0,1,2,3,4
Data = np.concatenate((data[pairs[j][0]], data[pairs[j][1]]), 0)
calculate_dists_line_by_line_frechet(Data, pair_idx=j, start_idx=0, end_idx=len(Data))

In [ ]:
j = 2
Data = np.concatenate((data[pairs[j][0]], data[pairs[j][1]]), 0)
calculate_dists_line_by_line_frechet(Data, pair_idx=j, start_idx=0, end_idx=len(Data))

In [ ]:
j = 3
Data = np.concatenate((data[pairs[j][0]], data[pairs[j][1]]), 0)
calculate_dists_line_by_line_frechet(Data, pair_idx=j, start_idx=0, end_idx=len(Data))

In [ ]:
j = 4
Data = np.concatenate((data[pairs[j][0]], data[pairs[j][1]]), 0)
calculate_dists_line_by_line_frechet(Data, pair_idx=j, start_idx=0, end_idx=len(Data))

## To stack the 1d generated vectors together to get the distance matrix of continuous Frechet distance we will use the following get_matrix function.

In [ ]:
def get_matrix(data1, data2, pair_idx, path):
    '''
    path is: 
    '/content/gdrive/My Drive/traj-dist/Calculated Distance Matrices (T-drive)/frechet lines '
    '''
    n = len(data1) + len(data2)
    matrix = np.zeros((n,n))
    for i in range(n-1):
        #matrix[i][i+1:] = np.array(pd.read_csv(path+str(i)+'.csv', header=None)).reshape(n-i-1,)
        matrix[i][i+1:] = np.loadtxt(path+str(pair_idx)+'/'+str(i)+'.csv')
    for i in range(1, n):
        for j in range(i):
            matrix[i][j] = matrix[j][i]
    np.savetxt(path[:-14]+'frechet '+str(pair_idx)+'.csv', matrix, delimiter=',')
    return matrix

# KNN with Continuous Frechet

In [ ]:
def KNN_with_dists(n_1, n_2, dists_names, paths_to_dists):
    '''path example: '/content/gdrive/My Drive/traj-dist/Calculated Distance Matrices (car-bus)/sspd.csv'
       dists_names: a list of distance names
       paths_to_dists: the paths list to the corresponding distancas (each path 
                       points out to the corresponding distance matrix)
       n_1: len(data_1)
       n_2: len(data_2)
       dist_name: the name of distance used to calculate sitance matrix 
       (the name is taken from a list above called metrics)'''

    train_errors = np.zeros(len(dists_names))
    test_errors = np.zeros(len(dists_names))

    I_1, J_1, y_train_1, y_test_1 = train_test_split(np.arange(n_1), 
                                                np.ones(n_1), test_size=0.3)
    I_2, J_2, y_train_2, y_test_2 = train_test_split(np.arange(n_1, n_1+n_2), 
                                                np.ones(n_2), test_size=0.3)
    labels = np.array([1]*n_1 + [0] * n_2)
    I = np.concatenate((I_1, I_2), 0)
    np.random.shuffle(I)
    J = np.concatenate((J_1, J_2), 0)
    np.random.shuffle(J)

    for i in range(len(dists_names)):

        dist_matrix = np.array(pd.read_csv(paths_to_dists[i], header=None))

        D_train = dist_matrix[I][:, I]
        D_test = dist_matrix[J][:,I]
        train_labels = labels[I]
        test_labels = labels[J]

        clf = KNeighborsClassifier(n_neighbors=5, metric='precomputed')
        
        #Train the model using the training sets
        clf.fit(D_train, list(train_labels))

        #Predict labels for train dataset
        train_pred = clf.predict(D_train)
        train_errors[i] = sum(train_labels != train_pred)/len(I)
        
        #Predict labels for test dataset
        test_pred = clf.predict(D_test)
        test_errors[i] = sum((test_labels != test_pred))/len(J)
        
    return train_errors, test_errors

In [ ]:
def KNN_average_error(data1, data2, num_trials, dists_names, paths_to_dists):

    '''dists_names: a list of distance names
       paths_to_dists: the paths list to the corresponding distancas (each path 
                       points out to the corresponding distance matrix)'''

    Start_time = time.time()

    train_errors = np.zeros((num_trials, len(dists_names)))
    test_errors = np.zeros((num_trials, len(dists_names)))

    for i in range(num_trials):
        tr_errors, ts_errors = KNN_with_dists(len(data1), len(data2), dists_names, paths_to_dists)
        train_errors[i] = tr_errors
        test_errors[i] = ts_errors

    train_error = np.mean(train_errors, axis=0)
    test_error = np.mean(test_errors, axis=0)
    std_test_error = np.std(test_errors, axis=0)

    Dict = {}
    for i in range(len(dists_names)):
        Dict[i+1] = [f"KNN with {dists_names[i]}", 
                     np.round(train_error[i], decimals = 4), 
                     np.round(test_error[i], decimals = 4), 
                     np.round(std_test_error[i], decimals = 4)]

    df = pd.DataFrame.from_dict(Dict, orient='index', columns=['Classifier',
                                'Train Error', 'Test Error', 'std'])
    print(colored(f"num_trials = {num_trials}", "blue"))
    print(colored(f'total time = {time.time() - Start_time}', 'green'))

    return (df, train_errors, test_errors, train_error, test_error)

In [ ]:
path = '/content/gdrive/My Drive/traj-dist/Calculated Distance Matrices (T-drive)/frechet lines '

In [ ]:
for j in range(len(pairs)):
    A = get_matrix(data[pairs[j][0]], data[pairs[j][1]], pair_idx=j, path=path)
    #print(A.shape, len(data[pairs[j][0]])+len(data[pairs[j][1]]))

In [ ]:
test_error = []
test_std = []
for j in range(len(pairs)):
    F = KNN_average_error(data[pairs[j][0]], data[pairs[j][1]], num_trials=50, 
                        dists_names=['frechet'], 
                        paths_to_dists=[path[:-14]+'frechet '+str(j)+'.csv'])
    print(colored(pairs[j], 'yellow'))
    print('pair:', F[0])
    print(colored('==========================================================', 'red'))
    test_error.append(np.array(F[0])[0][2])
    test_std.append(np.array(F[0])[0][3])
print(colored(f'mean test error: {np.mean(test_error)}', 'magenta'))
print(colored(f'mean std of error: {np.mean(test_std)}', 'blue'))

num_trials = 50
total time = 2.4297242164611816
[3 8]
pair:          Classifier  Train Error  Test Error     std
1  KNN with frechet       0.2523      0.3787  0.0366
num_trials = 50
total time = 2.5147180557250977
[7 9]
pair:          Classifier  Train Error  Test Error     std
1  KNN with frechet       0.2552      0.3838  0.0461
num_trials = 50
total time = 2.5965423583984375
[1 5]
pair:          Classifier  Train Error  Test Error     std
1  KNN with frechet       0.2435      0.3711  0.0477
num_trials = 50
total time = 2.259402275085449
[2 4]
pair:          Classifier  Train Error  Test Error     std
1  KNN with frechet       0.1274      0.1683  0.0295
num_trials = 50
total time = 2.540379762649536
[0 6]
pair:          Classifier  Train Error  Test Error     std
1  KNN with frechet       0.1541      0.1818  0.0314
mean test error: 0.29673999999999995
mean std of error: 0.038259999999999995


# KNN with LSH

In [ ]:
pip install trjtrypy

In [ ]:
from google.colab import files
files.upload()

#import KNN_with_LSH_class
from KNN_with_LSH_class import KNN_with_LSH

Saving KNN_with_LSH_class.py to KNN_with_LSH_class (1).py


In [ ]:
test_error_LSH = []
test_std_LSH = []
for j in range(len(pairs)):
    m, n = pairs[j]
    print(len(data[m])+len(data[n]))
    KNN_with_LSH_class = KNN_with_LSH(data[m], data[n], number_circles=20, num_trials=50)
    F = KNN_with_LSH_class.KNN_LSH_average_error()
    print(colored(pairs[j], 'yellow'))
    print('pair:', F[0])
    print(colored('==========================================================', 'red'))
    test_error_LSH.append(np.array(F[0])[0][2])
    test_std_LSH.append(np.array(F[0])[0][3])
print(colored(f'mean test error: {np.mean(test_error_LSH)}', 'magenta'))
print(colored(f'mean std of error: {np.mean(test_std_LSH)}', 'blue'))

259
num_trials = 50
total time = 4.845942497253418
[3 8]
pair:      Classifier  Train Error  Test Error     std
1  KNN with LSH       0.4564      0.4736  0.0634
262
num_trials = 50
total time = 5.11359429359436
[7 9]
pair:      Classifier  Train Error  Test Error    std
1  KNN with LSH       0.3048      0.4035  0.066
263
num_trials = 50
total time = 5.626368284225464
[1 5]
pair:      Classifier  Train Error  Test Error     std
1  KNN with LSH       0.3902      0.4395  0.0416
232
num_trials = 50
total time = 4.252983570098877
[2 4]
pair:      Classifier  Train Error  Test Error     std
1  KNN with LSH       0.2147      0.2606  0.0612
259
num_trials = 50
total time = 5.393928289413452
[0 6]
pair:      Classifier  Train Error  Test Error     std
1  KNN with LSH        0.178      0.2261  0.0496
mean test error: 0.36066
mean std of error: 0.05636
